In [39]:
pip install triton==3.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 5.7 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia

In [1]:
import torch
import triton
import triton.language as tl

from typing import Tuple

In [2]:
@triton.autotune(
    configs=[
        triton.Config({'BLOCK_D': 16}, num_warps=1),
        triton.Config({'BLOCK_D': 16}, num_warps=2),

        triton.Config({'BLOCK_D': 32}, num_warps=2),
        triton.Config({'BLOCK_D': 32}, num_warps=4),

        triton.Config({'BLOCK_D': 64}, num_warps=1),
        triton.Config({'BLOCK_D': 64}, num_warps=2),
        triton.Config({'BLOCK_D': 64}, num_warps=4),

        triton.Config({'BLOCK_D': 128}, num_warps=2),
        triton.Config({'BLOCK_D': 128}, num_warps=4),
        triton.Config({'BLOCK_D': 128}, num_warps=8),

        triton.Config({'BLOCK_D': 256}, num_warps=4),
        triton.Config({'BLOCK_D': 256}, num_warps=8),

        triton.Config({'BLOCK_D': 512}, num_warps=4),
        triton.Config({'BLOCK_D': 512}, num_warps=8),

        triton.Config({'BLOCK_D': 1024}, num_warps=8),
        triton.Config({'BLOCK_D': 1024}, num_warps=16),
    ],
    key=['HEAD_DIM', 'ROPE_DIM', 'NUM_ELEMENTS'],
)

@triton.jit
def _rope_fused_kernel(
    x_ptr, cos_ptr, sin_ptr, out_ptr,
    HEAD_DIM: tl.constexpr,
    ROPE_DIM: tl.constexpr,
    NUM_ELEMENTS: tl.constexpr,
    BLOCK_D: tl.constexpr,
):
    offs_d = tl.arange(0, BLOCK_D)
    offs = tl.program_id(axis=0) * BLOCK_D + offs_d

    is_in_bounds = offs < NUM_ELEMENTS
    x = tl.load(x_ptr + offs, mask=is_in_bounds, other=0.0)

    ROPE_OFFSET: tl.constexpr = HEAD_DIM - ROPE_DIM
    offs_rope = (offs_d % HEAD_DIM) - ROPE_OFFSET

    HALF_ROPE_DIM: tl.constexpr = ROPE_DIM // 2
    is_first_half = offs_rope < HALF_ROPE_DIM
    rope_partner = tl.gather(x, offs_d + tl.where(is_first_half, HALF_ROPE_DIM, -HALF_ROPE_DIM), axis=0)

    use_rope = offs_rope >= 0
    cos = tl.load(cos_ptr + offs_rope, mask=use_rope, other=0.0)
    sin = tl.load(sin_ptr + offs_rope, mask=use_rope, other=0.0)

    out = tl.where(use_rope, x * cos + tl.where(is_first_half, -1.0,  1.0) * rope_partner * sin, x)
    tl.store(out_ptr + offs, out, mask=is_in_bounds)

In [3]:
# @triton.autotune(
#     configs=[
#         triton.Config({'XBLOCK': 32}, num_warps=2),
#         triton.Config({'XBLOCK': 64}, num_warps=1),
#         triton.Config({'XBLOCK': 128}, num_warps=4),
#         triton.Config({'XBLOCK': 256}, num_warps=8),
#     ],
#     key=['xnumel'],
# )
# @triton.jit
# def triton_poi_fused_cat_0(in_ptr0, in_ptr1, in_ptr2, out_ptr0, xnumel, XBLOCK : tl.constexpr):
#     xnumel = 16777216
#     xoffset = tl.program_id(0) * XBLOCK
#     xindex = xoffset + tl.arange(0, XBLOCK)[:]
#     xmask = tl.full([XBLOCK], True, tl.int1)
#     x0 = (xindex % 64)
#     x1 = xindex // 64
#     x2 = xindex
#     tmp0 = x0
#     tmp1 = tl.full([1], 0, tl.int64)
#     tmp2 = tmp0 >= tmp1
#     tmp3 = tl.full([1], 16, tl.int64)
#     tmp4 = tmp0 < tmp3
#     tmp5 = tl.load(in_ptr0 + (64*x1 + (x0)), tmp4, eviction_policy='evict_last', other=0.0)
#     tmp6 = tmp0 >= tmp3
#     tmp7 = tl.full([1], 64, tl.int64)
#     tmp8 = tmp0 < tmp7
#     tmp9 = tl.load(in_ptr0 + (16 + 64*x1 + ((-16) + x0)), tmp6, eviction_policy='evict_last', other=0.0)
#     tmp10 = tl.load(in_ptr1 + ((-16) + x0), tmp6, eviction_policy='evict_last', other=0.0)
#     tmp11 = tmp9 * tmp10
#     tmp12 = (-16) + x0
#     tmp13 = tl.full([1], 0, tl.int64)
#     tmp14 = tmp12 >= tmp13
#     tmp15 = tl.full([1], 24, tl.int64)
#     tmp16 = tmp12 < tmp15
#     tmp17 = tmp16 & tmp6
#     tmp18 = tl.load(in_ptr0 + (40 + 64*x1 + ((-16) + x0)), tmp17, eviction_policy='evict_last', other=0.0)
#     tmp19 = -tmp18
#     tmp20 = tl.full(tmp19.shape, 0.0, tmp19.dtype)
#     tmp21 = tl.where(tmp17, tmp19, tmp20)
#     tmp22 = tmp12 >= tmp15
#     tmp23 = tl.full([1], 48, tl.int64)
#     tmp24 = tmp12 < tmp23
#     tmp25 = tmp22 & tmp6
#     tmp26 = tl.load(in_ptr0 + (16 + 64*x1 + ((-24) + ((-16) + x0))), tmp25, eviction_policy='evict_last', other=0.0)
#     tmp27 = tl.where(tmp16, tmp21, tmp26)
#     tmp28 = tl.load(in_ptr2 + ((-16) + x0), tmp6, eviction_policy='evict_last', other=0.0)
#     tmp29 = tmp27 * tmp28
#     tmp30 = tmp11 + tmp29
#     tmp31 = tl.full(tmp30.shape, 0.0, tmp30.dtype)
#     tmp32 = tl.where(tmp6, tmp30, tmp31)
#     tmp33 = tl.where(tmp4, tmp5, tmp32)
#     tl.store(out_ptr0 + (x2), tmp33, None)


In [4]:
def apply_rotary_pos_emb_triton(
    x: torch.Tensor,
    cos_sin: Tuple[torch.Tensor, torch.Tensor],
) -> torch.Tensor:
    if x.device.type != "cuda":
        raise RuntimeError("Triton kernel requires CUDA tensor")
    if x.dtype not in (torch.float16, torch.bfloat16, torch.float32):
        raise TypeError("x must be fp16, bf16, or fp32")

    cos, sin = cos_sin

    head_dim = x.size(-1)
    rope_dim = cos.size(-1)

    if rope_dim % 2:
        raise ValueError("rope_dim must be even")
    if rope_dim > head_dim:
        raise ValueError("rope_dim should be less than or equal to head_dim")

    out = torch.empty_like(x)
    x_numel = x.numel()

    grid = lambda META: (triton.cdiv(x_numel, META['BLOCK_D']),)

    _rope_fused_kernel[grid](
        x, cos, sin, out,
        head_dim,
        rope_dim,
        x_numel,
    )

    return out.view_as(x)

In [5]:
def apply_rotary_pos_emb(
    x: torch.Tensor, cos_sin: tuple[torch.Tensor, torch.Tensor]
) -> tuple[torch.Tensor, torch.Tensor]:
    cos, sin = cos_sin

    head_dim = x.size(-1)
    rope_dim = cos.size(-1)

    if head_dim == rope_dim:
        x = (x * cos) + (_rotate_half(x) * sin)
    elif rope_dim < head_dim:
        x_nope, x_rope = x.split((head_dim - rope_dim, rope_dim), dim=-1)
        x_rope = (x_rope * cos) + (_rotate_half(x_rope) * sin)
        x = torch.cat([x_nope, x_rope], dim=-1)
    else:
        raise ValueError("rope_dim should be less than head_dim")

    return x

def _rotate_half(x: torch.Tensor) -> torch.Tensor:
    x1, x2 = torch.chunk(x, 2, dim=-1)
    return torch.cat((-x2, x1), dim=-1)

In [6]:
if __name__ == "__main__":
    torch.manual_seed(0)
    batch_size = 32
    seq_len = 1024
    num_heads = 8
    head_dim = 64
    rope_dim = 48

    x = torch.randn(batch_size, seq_len, num_heads, head_dim, device="cuda", dtype=torch.float32)
    cos = torch.randn(rope_dim, device="cuda", dtype=torch.float32)
    sin = torch.randn_like(cos)

    ref = apply_rotary_pos_emb(x, (cos, sin))
    tri = apply_rotary_pos_emb_triton(x, (cos, sin))

    diff = (ref - tri).abs().max()
    print("max|diff| =", diff.item())

    ref = (apply_rotary_pos_emb)(x, (cos, sin))
    tri = (apply_rotary_pos_emb_triton)(x, (cos, sin))

    diff = (ref - tri).abs().max()
    print("max|diff| =", diff.item())

max|diff| = 9.5367431640625e-07
max|diff| = 9.5367431640625e-07


In [7]:
torch.manual_seed(42)
apply_rotary_pos_emb = (apply_rotary_pos_emb)
apply_rotary_pos_emb_triton = (apply_rotary_pos_emb_triton)

batch_size = 32
seq_len = 1024
num_heads = 8
head_dim = 64
rope_dim = 64

x = torch.randn(batch_size, seq_len, num_heads, head_dim, device="cuda", dtype=torch.float32)
cos = torch.randn(rope_dim, device="cuda", dtype=torch.float32)
sin = torch.randn_like(cos)

for _ in range(0):
    _ = apply_rotary_pos_emb(x, (cos, sin))
    _ = apply_rotary_pos_emb_triton(x, (cos, sin))

total_ref_time = 0.0
total_triton_time = 0.0
num_runs = 100

for i in range(num_runs):
    torch.manual_seed(i)

    x = torch.randn(batch_size, seq_len, num_heads, head_dim, device="cuda", dtype=torch.float32)
    cos = torch.randn(rope_dim, device="cuda", dtype=torch.float32)
    sin = torch.randn_like(cos)

    torch.cuda.synchronize()

    start_ref = torch.cuda.Event(enable_timing=True)
    end_ref = torch.cuda.Event(enable_timing=True)
    start_ref.record()
    ref = apply_rotary_pos_emb(x, (cos, sin))
    end_ref.record()
    torch.cuda.synchronize()
    ref_time = start_ref.elapsed_time(end_ref)
    total_ref_time += ref_time

    start_tri = torch.cuda.Event(enable_timing=True)
    end_tri = torch.cuda.Event(enable_timing=True)
    start_tri.record()
    tri = apply_rotary_pos_emb_triton(x, (cos, sin))
    end_tri.record()
    torch.cuda.synchronize()
    triton_time = start_tri.elapsed_time(end_tri)
    total_triton_time += triton_time

    print(triton_time)
    print()

    diff = (ref - tri).abs().max()

average_ref_time = total_ref_time / num_runs
average_triton_time = total_triton_time / num_runs

print("-" * 30)
print(f"Average ref time over {num_runs} runs: {average_ref_time:.4f} ms")
print(f"Average triton time over {num_runs} runs: {average_triton_time:.4f} ms")

2008.0614013671875

0.6913279891014099

0.6683520078659058

0.654591977596283

0.6594560146331787

0.653439998626709

0.6574079990386963

0.6570879817008972

0.6480960249900818

0.6518719792366028

0.6631039977073669

0.6541119813919067

0.6512960195541382

0.6559360027313232

0.6553599834442139

0.647167980670929

0.6500160098075867

0.6635519862174988

0.6922240257263184

0.6522240042686462

0.662559986114502

0.6615039706230164

0.6594560146331787

0.6512640118598938

0.6600639820098877

0.6556159853935242

0.6601920127868652

0.657696008682251

0.6525760293006897

0.6711999773979187

0.6553599834442139

0.6528639793395996

0.6573759913444519

0.6553599834442139

0.6512640118598938

0.8942080140113831

0.6574079990386963

0.6512640118598938

0.6553599834442139

0.6531199812889099

0.6461759805679321

0.646336019039154

0.650111973285675

0.6647359728813171

0.6470080018043518

0.7101439833641052

0.7275840044021606

0.7017920017242432

0.7167999744415283

0.7043520212173462

0.66134